# 🐸 Project Euler Problem 329 — Prime Frog

## Nome: Morsinaldo de Azevedo Medeiros

Susan has a prime frog.  
Her frog is jumping around over 500 squares numbered 1 to 500. He can only jump one square to the left or to the right, with equal probability, and he cannot jump outside the range [1;500].  
(If it lands at either end, it automatically jumps to the only available square on the next move.)

When he is on a square with a prime number on it, he croaks 'P' (PRIME) with probability 2/3 or 'N' (NOT PRIME) with probability 1/3 just before jumping to the next square.  
When he is on a square with a number on it that is not a prime he croaks 'P' with probability 1/3 or 'N' with probability 2/3 just before jumping to the next square.  

Given that the frog's starting position is random with the same probability for every square, and given that she listens to his first 15 croaks, what is the probability that she hears the sequence:

**PPPPNNPPPNPPNPN**

Give your answer as a fraction p/q in reduced form.

In [3]:
# Imports e sequência
import numpy as np
from fractions import Fraction
import sympy as sp

SEQ = "PPPPNNPPPNPPNPN"   # sequência observada (15 símbolos)
N = 500                   # número de posições

## 🔢 Conjunto de primos em 1..500

Usaremos `sympy.primerange` para obter o conjunto de primos.  
Trataremos o índice da posição como 1-based (i.e., posição i corresponde ao número i).

In [4]:
primes = set(sp.primerange(1, N+1))
len(primes), 1 in primes  # 1 NÃO é primo (checagem)

(95, False)

## 📥 Distribuição inicial (prior)

A distribuição inicial é **uniforme** sobre as 500 posições.  
Para obter a fração reduzida, usamos `Fraction` para manter aritmética exata.

In [5]:
# vetor de probabilidades sobre posições 1..N (usaremos dtype=object)
prob = np.array([Fraction(1, N)] * N, dtype=object)
# checagem: soma deve ser 1
sum(prob)

Fraction(1, 1)

## 🔊 Passo de fala

Para cada símbolo da sequência:
- multiplicamos cada entrada `prob[i]` por `P(emitir símbolo | posição i)`;
- a probabilidade de falar o quê depende se `i+1` é primo (pois o vetor é 0-based, a posição é i+1).

Regras:
- Se primo: `P(P)=2/3`, `P(N)=1/3`;
- Se não primo: `P(P)=1/3`, `P(N)=2/3`.

In [6]:
def emission_step(prob_vec, symbol):
    out = prob_vec.copy()
    if symbol == "P":
        for i in range(N):
            if (i+1) in primes:
                out[i] *= Fraction(2, 3)
            else:
                out[i] *= Fraction(1, 3)
    else:  # symbol == "N"
        for i in range(N):
            if (i+1) in primes:
                out[i] *= Fraction(1, 3)
            else:
                out[i] *= Fraction(2, 3)
    return out

## 🔁 Passo de transição

Após falar, o sapo se move:
- da posição 1 → 2 com probabilidade 1;
- da posição 500 → 499 com probabilidade 1;
- das posições 2..499: 1/2 para a esquerda (i-1) e 1/2 para a direita (i+1).

In [7]:
def transition_step(prob_vec):
    nxt = np.zeros(N, dtype=object)
    half = Fraction(1, 2)

    # posição 1 (índice 0) vai forçado para 2 (índice 1)
    nxt[1] += prob_vec[0]

    # posição 500 (índice 499) vai forçado para 499 (índice 498)
    nxt[N-2] += prob_vec[N-1]

    # posições internas
    for i in range(1, N-1):
        nxt[i-1] += prob_vec[i] * half
        nxt[i+1] += prob_vec[i] * half

    return nxt

## ⏩ Loop *forward* (emissão + transição)

Aplicamos, para cada símbolo `ch` da sequência, primeiro a fala e depois a transição.

In [8]:
p = prob.copy()
for ch in SEQ:
    p = emission_step(p, ch)
    p = transition_step(p)

# probabilidade final da sequência = soma das probabilidades após o último passo
ans = sum(p)
ans

Fraction(199740353, 29386561536000)

## ✅ Resultado final

`ans` já é uma fração reduzida `p/q`.  
Abaixo, mostramos também a representação decimal (apenas para referência).

In [9]:
print("p/q =", ans)
print("decimal ≈", float(ans))

p/q = 199740353/29386561536000
decimal ≈ 6.7969964010695205e-06


![Alt text](./figures/project_euler_329_answered.png "Project Euler 329 Answered")